# Import required functions

In [26]:
import pandas as pd

from helper import (
    create_connection_string,
    create_city_df,
    scrape_city_soups_from_wikipedia,
    create_geo_df_from_soups,
    create_population_df_from_soups,
    get_airports
)

# Load config

In [28]:
with open("config.json", "r") as f:
    config = json.load(f)
    db_config = config["db_config"]
    api_config = config["api_config"]
    print("Config loaded.")

Config loaded.


# Create connection string

In [30]:
connection_string = create_connection_string(
    db_config["schema"],
    db_config["host"],
    db_config["user"],
    db_config["password"],
    db_config["port"]
)

# Define city baseline

In [32]:
cities = [
    ('Berlin', 'DE'),
    ('Vienna', 'AT'),
    ('New York City', 'US'),
    ('Munich', 'DE'),
    ('Hamburg', 'DE'),
    ('Paris', 'FR'),
    ('London', 'GB'),
    ('Tokyo', 'JP'),
    ('Sydney', 'AU'),
    ('Rio de Janeiro', 'BR'),
    ('Moscow', 'RU'),
    ('Dubai', 'AE'),
    ('Toronto', 'CA'),
    ('Cologne', 'DE'),
    ('Amsterdam', 'NL')
]

# Create cities table and send to SQL

In [34]:
df_cities = create_city_df(cities)
df_cities

,city_name,city_country_code
0,Berlin,DE
1,Vienna,AT
2,New York City,US
3,Munich,DE
4,Hamburg,DE
5,Paris,FR
6,London,GB
7,Tokyo,JP
8,Sydney,AU
9,Rio de Janeiro,BR


In [35]:
df_cities.to_sql(
    'cities',
    if_exists='append',
    con=connection_string,
    index=False
)

15

# Receive cities table from SQL with IDs

In [37]:
cities_from_sql = pd.read_sql("cities", con=connection_string)
cities_from_sql

,city_id,city_name,city_country_code
0,1,Berlin,DE
1,2,Vienna,AT
2,3,New York City,US
3,4,Munich,DE
4,5,Hamburg,DE
5,6,Paris,FR
6,7,London,GB
7,8,Tokyo,JP
8,9,Sydney,AU
9,10,Rio de Janeiro,BR


# Scrape Wikipedia of cities

In [39]:
city_soups = scrape_city_soups_from_wikipedia(cities)
print(len(city_soups))

15


# Create geodata table, enrich with city IDs and send to SQL

In [41]:
df_city_geodata = create_geo_df_from_soups(cities, city_soups)
df_city_geodata

,city_latitude,city_longitude
0,52.516667,13.400000
1,48.200000,16.366667
2,40.700000,-74.000000
3,48.133333,11.566667
4,53.550000,10.000000
5,48.850000,2.350000
6,51.500000,-0.116667
7,35.683333,139.683333
8,-33.866667,151.200000
9,-22.900000,-43.200000


In [42]:
df_city_geodata = pd.concat([df_cities, df_city_geodata], axis=1).merge(
    cities_from_sql,
    on = "city_name",
    how="left"
)[['city_id', 'city_longitude', 'city_latitude']]
df_city_geodata

,city_id,city_longitude,city_latitude
0,1,13.400000,52.516667
1,2,16.366667,48.200000
2,3,-74.000000,40.700000
3,4,11.566667,48.133333
4,5,10.000000,53.550000
5,6,2.350000,48.850000
6,7,-0.116667,51.500000
7,8,139.683333,35.683333
8,9,151.200000,-33.866667
9,10,-43.200000,-22.900000


In [43]:
df_city_geodata.to_sql(
    'geodata',
    if_exists='append',
    con=connection_string,
    index=False
)

15

# Create populations table, enrich with city IDs and send to SQL

In [45]:
df_city_population = create_population_df_from_soups(cities, city_soups)
df_city_population

,population,timestamp_population
0,3596999,2025-05-28
1,2028499,2025-05-28
2,8804190,2025-05-28
3,1510378,2025-05-28
4,1964021,2025-05-28
5,2048472,2025-05-28
6,8866180,2025-05-28
7,14187176,2025-05-28
8,5557233,2025-05-28
9,6211223,2025-05-28


In [46]:
df_city_population = pd.concat([df_cities, df_city_population], axis=1).merge(
    cities_from_sql,
    on = "city_name",
    how="left"
)[['city_id', 'population', 'timestamp_population']]
df_city_population

,city_id,population,timestamp_population
0,1,3596999,2025-05-28
1,2,2028499,2025-05-28
2,3,8804190,2025-05-28
3,4,1510378,2025-05-28
4,5,1964021,2025-05-28
5,6,2048472,2025-05-28
6,7,8866180,2025-05-28
7,8,14187176,2025-05-28
8,9,5557233,2025-05-28
9,10,6211223,2025-05-28


In [47]:
df_city_population.to_sql(
    'populations',
    if_exists='append',
    con=connection_string,
    index=False
)

15

# Create cities_airports and airports table, enrich with city IDs and send to SQL

In [49]:
df_city_airports = get_airports(
    df_city_geodata['city_id'].values,
    df_city_geodata['city_latitude'].values,
    df_city_geodata['city_longitude'].values,
    api_config
)
df_city_airports

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon,city_id,localCode
0,EDDT,TXL,Berlin -Tegel,-Tegel,Berlin,DE,Europe/Berlin,52.559700,13.287699,1,NaN
1,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.351390,13.493889,1,NaN
2,LOWW,VIE,Vienna Schwechat,Schwechat,Vienna,AT,Europe/Vienna,48.110300,16.569700,2,NaN
3,KLGA,LGA,New York La Guardia,La Guardia,New York,US,America/New_York,40.777200,-73.872600,3,NaN
4,KEWR,EWR,Newark Liberty,Liberty,Newark,US,America/New_York,40.692500,-74.168700,3,NaN
5,KTEB,TEB,Teterboro,Teterboro,Teterboro,US,America/New_York,40.850100,-74.060800,3,NaN
6,KJFK,JFK,New York John F Kennedy,John F Kennedy,New York,US,America/New_York,40.639800,-73.778900,3,NaN
7,KCDW,CDW,Caldwell Essex County,Essex County,Caldwell,US,America/New_York,40.875200,-74.281400,3,NaN
8,KMMU,MMU,Morristown Municipal,Municipal,Morristown,US,America/New_York,40.799400,-74.414900,3,NaN
9,KHPN,HPN,White Plains Westchester County,Westchester County,White Plains,US,America/New_York,41.067000,-73.707600,3,NaN


In [50]:
df_airports = df_city_airports.loc[:, ['icao', 'name']].rename(
    columns={
    'icao' : 'airport_icao',
    'name' : 'airport_name'
    }
)
df_airports

,airport_icao,airport_name
0,EDDT,Berlin -Tegel
1,EDDB,Berlin Brandenburg
2,LOWW,Vienna Schwechat
3,KLGA,New York La Guardia
4,KEWR,Newark Liberty
5,KTEB,Teterboro
6,KJFK,New York John F Kennedy
7,KCDW,Caldwell Essex County
8,KMMU,Morristown Municipal
9,KHPN,White Plains Westchester County


In [51]:
df_airports.to_sql(
    'airports',
    if_exists='append',
    con=connection_string,
    index=False
)

39

In [52]:
df_cities_airports = df_city_airports.loc[:, ['city_id', 'icao']].rename(
    columns={'icao' : 'airport_icao'}
)
df_cities_airports

,city_id,airport_icao
0,1,EDDT
1,1,EDDB
2,2,LOWW
3,3,KLGA
4,3,KEWR
5,3,KTEB
6,3,KJFK
7,3,KCDW
8,3,KMMU
9,3,KHPN


In [53]:
df_cities_airports.to_sql(
    'cities_airports',
    if_exists='append',
    con=connection_string,
    index=False
)

39